#### надо установить opencv для python, если не установлен.
#### pip(pip3) install opencv-python

In [17]:
import cv2
import numpy as np

Пример получения вектора признаков для изображения

In [18]:
# далее задаются параметры HOG-a, они выставлены как в статье.
win_size = (64,128) 
block_size = (16,16)
block_stride = (8,8)
cell_size = (8,8)
nbins = 9
deriv_aperture = 1
win_sigma = -1
histogram_norm_type = 0
l2_hys_threshold = 2.0000000000000001e-01
gamma_correction = 0
nlevels = 64
hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,nbins,deriv_aperture,win_sigma,
                        histogram_norm_type,l2_hys_threshold,gamma_correction,nlevels)


В базе INRIA размер изображений содержащих пешеходов  - 96х160(train) и 70x134(test). Человек расположен по центру.

Вам нужно получить вектор признаков(дескриптор) для окна 64х128. По умолчанию compute() строит дескрипторы для всех окон размером 64х128 методом скользящего окна, с шагом win_stride по обеим осям. Но если вам нужно получить дескриптор только для одного окна 64х128, то у метода есть аргумент locations, в котором можно передать координаты верхнего левого угла интересующего вас окна (либо список таких координат)

Например, для изображений из train извлечение центрального окна 64х128 выглядит следующим образом:

In [20]:
img = cv2.imread("crop_000010.png")
height, width = img.shape[:2]
locations = [((width-64)//2, (height-128)//2)]
descriptor = hog.compute(img, locations=locations)
descriptor.size

3780

Изображения фона(отрицательные примеры)  - изображения произвольного размера. В обучающей выборке их 1218. Авторы HOG для обучения выбирали рандомно 10 окон 64х128 из каждого изображения  - всего 12180 примеров, не содержащих пешеходов.

Для этого, например, можно посчитать дескрипторы по всему изображению с win_stride=(4,4) и потом рандомно выбрать 10 из них.

Такие манипуляции вам надо проделать в цикле со всеми изображениями из train: для изображений из папки pos/ выбирать центральное окно, а из папки neg/ - 10 рандомных окон. Собрать их в np.array  или список, как вам будет удобно ну и в файл сохранить тоже.

Для того чтобы получить ответ svm классификатора у класса HOGDescriptor есть метод detect(). Этот метод вам понадобится, чтобы строить кривые качества (DET кривые). Метод detect() работает похожим образом с compute().
Только перед его использованием нужно задать веса SVM:

    hog.setSVMDetector(hog.getDefaultPeopleDetector()) - используется веса из opencv для HOG, обученного на INRIA
В остальном поробуйте сами с detect() разобраться. Если не получится - пишите.

Гайд по построению DET кривых позже добавлю

# Выделим признаки на позитивных картинках

In [31]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
def imHOG(path_to_image):
    img = cv2.imread(path_to_image)
    height, width = img.shape[:2]
    locations = [((width-64)//2, (height-128)//2)]
    descriptor = hog.compute(img, locations=locations)
    return descriptor

In [32]:
import pandas as pd

images = open("pos.lst")
string = images.read()
images = string.split("\n")
for i in range(len(images)):
    images[i] = images[i][10:]
images = images[:-1]

In [33]:
pos = []
for i in range(len(images)):
    #desc_file.write(str(imHOG(images[i]))+"\n")
    descriptor_current = imHOG(images[i])
    pos.append(descriptor_current)

In [39]:
images = open("pos_test.lst")
string = images.read()
images = string.split("\n")
for i in range(len(images)):
    images[i] = images[i][9:]
images = images[:-1]


In [40]:
pos_test = []
for i in range(len(images)):
    #desc_file.write(str(imHOG(images[i]))+"\n")
    descriptor_current = imHOG(images[i])
    pos_test.append(descriptor_current)

# На негативных

In [43]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
def imHOG_neg(path_to_image):
    img = cv2.imread(path_to_image)
    win_stride = (4,4)
    #получаем дескрипторы изображения и приводим их к рамеру
    #(кол-во окон на изображении)х(рамер дескриптора одного окна)
    #рамер дексриптора для параметров HOG как в статье - 3780
    descriptors = hog.compute(img, win_stride).reshape(-1,3780)
    indexes = np.random.randint(descriptors.shape[0], size=10)
    ten_random_samples = descriptors[indexes]
    return ten_random_samples

In [44]:
images_neg = open("neg.lst")
string_neg = images_neg.read()
images_neg = string_neg.split("\n")
for i in range(len(images_neg)):
    images_neg[i] = images_neg[i][10:]
images_neg = images_neg[:-1]

In [45]:
neg = []
for i in range(len(images_neg)):
    desc_hog = imHOG(images_neg[i])
    neg.append(descriptor_current)

In [50]:
images_neg = open("neg_test.lst")
string_neg = images_neg.read()
images_neg = string_neg.split("\n")
for i in range(len(images_neg)):
    images_neg[i] = images_neg[i][9:]
images_neg = images_neg[:-1]

In [51]:
neg_test = []
for i in range(len(images_neg)):
    desc_hog = imHOG(images_neg[i])
    neg_test.append(descriptor_current)

# Решейпим полученные дескриаторы pos, pos_test, neg, neg_test

In [240]:
pos = np.array(pos)
pos_test = np.array(pos_test)
neg = np.array(neg)
neg_test = np.array(neg_test)



def reshape(matrix):
    l = matrix.shape[0]
    new_matrix = []
    cur = []
    for i in range(l):
        for j in range(3780):
            cur.append(matrix[i][j][0])
        new_matrix.append(cur)
        cur = []
    return new_matrix




In [82]:
neg_reshape = reshape(neg)
pos_reshape = reshape(pos)
pos_test_reshape = reshape(pos_test)
neg_test_reshape = reshape(neg_test)
neg_reshape = reshape_9_30_14(neg_reshape)
pos_reshape = reshape_9_30_14(pos_reshape)
pos_test_reshape = reshape_9_30_14(pos_test)
neg_test_reshape = reshape_9_30_14(neg_test_reshape)

### В каждом массиве хранятся векторы длины 3180. Построим сверточную нейронную сеть.

In [83]:
from matplotlib.pyplot import imshow

In [242]:

def reshape(matrix):
    l = matrix.shape[0]
    new_matrix = []
    cur = []
    for i in range(l):
        for j in range(3780):
            cur.append(matrix[i][j][0])
        new_matrix.append(cur)
        cur = []
    return new_matrix
def res_matr(descriptors):
    pos = np.array(descriptors)
    pos_reshape = reshape(pos)
    matr=[]
    for i in range(0,len(pos_reshape)):
        
        m_i=[]
        for j in range(0,15):
            q1=[]
            q2=[]
            for k in range(0,7):
                q1.append(pos_reshape[i][36*j*7+2*k*18:36*j*7+2*k*18+9])
                q1.append(pos_reshape[i][36*j*7+2*k*18+9:36*j*7+2*k*18+18])
                q2.append(pos_reshape[i][36*j*7+2*k*18+18:36*j*7+2*k*18+27])
                q2.append(pos_reshape[i][36*j*7+2*k*18+27:36*j*7+2*k*18+36])
            m_i.append(q1)
            m_i.append(q2)
        matr.append(m_i)
    return matr


def image_descr(mas,q,l):
    r=[]
    for i in range(0,30):
        for j in range (0,14):
            r=r+[mas[q][i][j][l]]
    r=np.array(r).reshape(14,30)
    #plt.imshow(r)
    return r

def reshape_9_30_14(mas):
    W=[]
    for i in range(len(mas)):
        W_i=[]
        for k in range(0,9):
            r=image_descr(mas,i,k)
            W_i.append(r)
        W.append(W_i)
    return W



In [243]:
neg_f=res_matr(neg)
pos_f=res_matr(pos)
neg_test_f=res_matr(neg_test)
pos_test_f=res_matr(pos_test)
neg_reshape_9 = reshape_9_30_14(neg_f)
pos_reshape_9 = reshape_9_30_14(pos_f)
pos_test_reshape_9 = reshape_9_30_14(pos_test_f)
neg_test_reshape_9 = reshape_9_30_14(neg_test_f)

In [ ]:
neg_reshape_9[1]

In [79]:
import numpy as np
import pandas as pd
import torch
from IPython.display import clear_output
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [268]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable




class CNNClassifier(nn.Module):
    """Custom module for a simple convnet classifier"""
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(9, 18, kernel_size=3)
        self.conv2 = nn.Conv2d(18, 36, kernel_size=3)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(3*144, 40)
        self.fc2 = nn.Linear(40, 2)
    
    def forward(self, x):
        # input is 30x14x9
        # conv1(kernel=3, filters=9) 30x14x18 -> 28x12x18
        # max_pool(kernel=2) 28x12x18 -> 14x6x18
        
        # Do not be afraid of F's - those are just functional wrappers for modules form nn package
        # Please, see for yourself - http://pytorch.org/docs/_modules/torch/nn/functional.html
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        
        # conv2(kernel=3, filters=36) 14x6x18 -> 12x4x36
        # max_pool(kernel=2) 12x4x36 -> 6x2x36
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        
        # flatten 6x2x36 = 3*144
        x = x.view(-1, 3*144)
        
        # 3*144 -> 40
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        # 40 -> 2
        x = self.fc2(x)
        
        # transform to logits
        return F.log_softmax(x)

# create classifier and optimizer objects
clf = CNNClassifier()
opt = torch.optim.Adam(clf.parameters(), lr=1e-3)

loss_history = []
acc_history = []
avg_loss = []

def train(epoch):
    clf.train() # set model in training mode (need this because of dropout)
    
    # dataset API gives us pythonic batching 
    for batch_id in range(60):
        #positive
        data = Variable(torch.tensor(pos_f[10*batch_id:10*(batch_id+1)], dtype=torch.float32))
        data = torch.reshape(data, [10,9,30,14])
        target = Variable(torch.tensor(np.ones(10), dtype=torch.long))
        target = torch.reshape(target, [10])
        # forward pass, calculate loss and backprop!
        opt.zero_grad()
        preds = clf(data)
        loss = F.nll_loss(preds, target)
        loss.backward()
        loss_history.append(loss.data[0])
        opt.step()
        
        #negative
        data = Variable(torch.tensor(neg_f[10*batch_id:10*(batch_id+1)], dtype=torch.float32))
        data = torch.reshape(data, [10,9,30,14])
        target = Variable(torch.tensor(np.zeros(10), dtype=torch.long))
        target = torch.reshape(target, [10])
        # forward pass, calculate loss and backprop!
        opt.zero_grad()
        preds = clf(data)
        loss = F.nll_loss(preds, target)
        loss.backward()
        loss_history.append(loss.data[0])
        opt.step()
        
        if batch_id % 10 == 0:
            print(loss.data[0])
    
    
    
def test(epoch):
    clf.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for batch_id in range(20):
        #negative
        data = Variable(torch.tensor(neg_test_f[10*batch_id:10*(batch_id+1)], dtype=torch.float32))
        data = torch.reshape(data, [10,9,30,14])
        target = Variable(torch.tensor(np.zeros(10), dtype=torch.long))
        target = torch.reshape(target, [10])
        
        output = clf(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        print("Neg {}".format(pred), output)
        correct += pred.eq(target.data).cpu().sum()
        #positive
        data = Variable(torch.tensor(pos_test_f[10*batch_id:10*(batch_id+1)], dtype=torch.float32))
        data = torch.reshape(data, [10,9,30,14])
        target = Variable(torch.tensor(np.ones(10), dtype=torch.long))
        target = torch.reshape(target, [10])
        
        output = clf(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        print("Pos {}".format(pred), output)
        correct += pred.eq(target.data).cpu().sum()
        
        

    test_loss = test_loss
    test_loss /= 200 # loss function already averages over batch size
    accuracy = 100. * correct / 200
    acc_history.append(accuracy)
    avg_loss.append(test_loss)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, 400,
        accuracy))



In [269]:
train(0)
test(0)

/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(0.7221)
tensor(0.7147)
tensor(0.7531)
tensor(0.7688)
tensor(0.6528)
tensor(0.5046)


/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:115: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Neg tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([[-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364]], grad_fn=<LogSoftmaxBackward>)
Pos tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([[-3.8455, -0.0216],
        [-1.6307, -0.2179],
        [-3.2741, -0.0386],
        [-2.5626, -0.0802],
        [-2.8019, -0.0626],
        [-2.4027, -0.0948],
        [-3.7506, -0.0238],
        [-1.3505, -0.2999],
        [-4.2611, -0.0142],
        [-1.8704, -0.1673]], grad_fn=<LogSoftmaxBackward>)
Neg tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([[-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364

Pos tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([[-1.7343, -0.1942],
        [-3.6289, -0.0269],
        [-2.2430, -0.1122],
        [-3.1362, -0.0444],
        [-3.3188, -0.0369],
        [-2.1394, -0.1252],
        [-1.8391, -0.1731],
        [-2.6434, -0.0738],
        [-2.5659, -0.0800],
        [-2.4560, -0.0897]], grad_fn=<LogSoftmaxBackward>)
Neg tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([[-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364],
        [-0.2715, -1.4364]], grad_fn=<LogSoftmaxBackward>)
Pos tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([[-3.6756, -0.0257],
        [-1.3906, -0.2863],
        [-3.7353, -0.0242],
        [-2.6463, -0.0736],
        [-2.7145, -0.0685],
        [-2.6250, -0.0752],
        [-2.6769, -0.0713],
        [-2.1233, -0.1274],
        [-2.0590, -0.1365

In [256]:
data = Variable(torch.tensor(pos_f[0:10], dtype=torch.float32))
#data = torch.reshape(data, [10,30,14,9])
data = torch.reshape(data, [10,9,30,14])
print(data[0][1][5][5], pos_f[0][5][5][1])

tensor(0.3211) 0.18128979


In [257]:
for epoch in range(0, 10):
    print("Epoch %d" % epoch)
    train(epoch)
    test(epoch)

Epoch 0
tensor(0.2460)
tensor(0.1251)


/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(0.0713)
tensor(0.0133)
tensor(0.0033)
tensor(0.0165)


/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/tomininvladislav/env/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0008, Accuracy: 400/400 (200%)

Epoch 1
tensor(0.0153)
tensor(0.0058)
tensor(0.0133)
tensor(0.0047)
tensor(0.0038)
tensor(0.0194)

Test set: Average loss: 0.0002, Accuracy: 400/400 (200%)

Epoch 2
tensor(0.0030)
tensor(0.0040)
tensor(0.0066)
tensor(0.0049)
tensor(0.0015)
tensor(0.0014)

Test set: Average loss: 0.0000, Accuracy: 400/400 (200%)

Epoch 3
tensor(0.0052)
tensor(0.0011)
tensor(0.0009)
tensor(0.0038)
tensor(0.0046)
tensor(0.0009)

Test set: Average loss: 0.0000, Accuracy: 400/400 (200%)

Epoch 4
tensor(0.0035)
tensor(0.0009)
tensor(0.0033)
tensor(0.0006)
tensor(0.0001)
tensor(0.0000)

Test set: Average loss: 0.0000, Accuracy: 400/400 (200%)

Epoch 5
tensor(0.0020)
tensor(0.0010)
tensor(0.0001)
tensor(0.0004)
tensor(0.0006)
tensor(0.0077)

Test set: Average loss: 0.0000, Accuracy: 400/400 (200%)

Epoch 6
tensor(0.0050)
tensor(0.0016)
tensor(0.0001)
tensor(0.0046)
tensor(0.0020)
tensor(0.0132)

Test set: Average loss: 0.0000, Accuracy: 400/400 (200%)
